<a href="https://colab.research.google.com/github/marisbotero/poesIA/blob/main/%F0%9F%92%9CGeneraci%C3%B3n_de_texto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Generación de textos con 🤗 Transformers
Cómo generar texto con diferentes métodos de descodificación.


- Cargar y explorar los datos


In [ ]:
!pip install transformers datasets

In [2]:
from IPython.display import HTML
    
def display_df(df, max_cols=15, header=True, index=True):
    return display(HTML(df.to_html(header=header,index=index, max_cols=max_cols)))

In [3]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "mrm8488/spanish-gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Añadimos el token EOS como token PAD para evitar warnings
model = AutoModelForCausalLM.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id).to(device)

Downloading:   0%|          | 0.00/226 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/846k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/505k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [4]:
def model_size(model):
    return sum(t.numel() for t in model.parameters())

print(f"Tamaño del GPT español: {model_size(model)/1000**2:.1f}M parámetros")

Tamaño del GPT español: 124.4M parámetros


In [7]:
import pandas as pd

# Aqui es el "prompt" para continuar
input_txt = "la magia de las cosas pequeñas "

input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 12
choices_per_step = 5

with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)
        # Seleccionar los logits del primer batch y del último token y aplicar softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)
        # Almacenar las tokens con mayores probabilidades
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (
                f"{tokenizer.decode(token_id)} ({100 * token_prob:.2f}%)"
            )
            iteration[f"Choice {choice_idx+1}"] = token_choice
        # Añadir el siguiente token previsto a los inputs
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)

display_df(pd.DataFrame.from_records(iterations), index=None)

Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
la magia de las cosas pequeñas,........ (7.54%),{\ (6.61%),ue (5.40%),} (5.28%),� (3.88%)
la magia de las cosas pequeñas........,.. (17.67%),... (17.39%),...... (14.42%),.... (13.45%),..... (5.21%)
la magia de las cosas pequeñas..........,.. (10.29%),... (6.64%),- (2.19%),Y (2.11%),¿ (1.99%)
la magia de las cosas pequeñas............,.. (38.85%),... (4.58%),y (4.22%),que (2.57%),es (2.24%)
la magia de las cosas pequeñas..............,.. (50.93%),y (3.46%),es (3.18%),... (2.48%),que (2.21%)
la magia de las cosas pequeñas................,.. (53.35%),es (3.90%),y (3.53%),que (2.08%),... (1.63%)
la magia de las cosas pequeñas..................,.. (57.45%),es (3.81%),y (3.43%),que (1.76%),... (1.24%)
la magia de las cosas pequeñas....................,.. (64.40%),y (3.07%),es (3.00%),que (1.26%),... (1.06%)
la magia de las cosas pequeñas......................,.. (71.79%),y (2.63%),es (2.09%),... (0.96%),que (0.84%)
la magia de las cosas pequeñas........................,.. (79.26%),y (2.04%),es (1.32%),... (0.85%),que (0.51%)


In [8]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_length=20)
print(tokenizer.decode(output[0]))

la magia de las cosas pequeñas................................


In [ ]:
max_length = 128
input_txt ="""En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios \
que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. \
Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban \
un inglés perfecto. """
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, 
                               do_sample=False)
print(tokenizer.decode(output_greedy[0]))

In [10]:
tokenizer.decode(output_greedy[0])

'En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇ'

In [11]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

In [12]:
def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [13]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇEl idioma de los unicornios!ˇ

log-prob: -20.17


In [14]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. Ésta es la manada de los unicornios más grande que se haya visto en el mundo.La manada de los unicornios más grande que se haya visto en el mundo.La manada de los unicornios más grande que se haya visto en el mundo.La manada de los unicornios más grande que se haya visto en el mundo.La manada

log-prob: -44.11


In [15]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, 
                             do_sample=False, no_repeat_ngram_size=2)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")


En un hallazgo sorprendente, los científicos descubrieron una manada de unicornios que vivía en un valle remoto, hasta ahora inexplorado, en la cordillera de los Andes. Más sorprendente aún para los investigadores fue el hecho de que los unicornios hablaban un inglés perfecto. Ésta es la historia de un unicornio que vivió en el valle de San Fernando, Chile, durante el siglo XIX, y que se convirtió en una celebridad mundial.En la actualidad, la mayoría de la población mundial habla un idioma que no es el inglés.El idioma más hablado en todo el mundo, el español, es una de las lenguas más habladas

log-prob: -95.42
